In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_diagonal_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,9381.87,-0.14,-0.03,-0.02,0.00,-0.00,0.00,-0.00
3,-0.14,1374.58,0.01,-0.01,0.00,-0.00,-0.00,0.00
4,-0.03,0.01,583.44,-0.00,0.01,0.00,-0.00,-0.00
5,-0.02,-0.01,-0.00,161.03,-0.00,-0.00,0.00,0.00
6,0.00,0.00,0.01,-0.00,39.28,0.00,0.00,0.00
7,-0.00,-0.00,0.00,-0.00,0.00,9.06,-0.00,-0.00
8,0.00,-0.00,-0.00,0.00,0.00,-0.00,1.97,0.00
9,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.35


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00004,-0.00001,-0.00002,0.00000,-0.00001,0.00001,-0.00003
3,-0.00004,1.00000,0.00001,-0.00002,0.00001,-0.00001,-0.00001,0.00000
4,-0.00001,0.00001,1.00000,-0.00001,0.00005,0.00005,-0.00001,-0.00005
5,-0.00002,-0.00002,-0.00001,1.00000,-0.00001,-0.00001,0.00002,0.00000
6,0.00000,0.00001,0.00005,-0.00001,1.00000,0.00002,0.00000,0.00006
7,-0.00001,-0.00001,0.00005,-0.00001,0.00002,1.00000,-0.00001,-0.00001
8,0.00001,-0.00001,-0.00001,0.00002,0.00000,-0.00001,1.00000,0.00003
9,-0.00003,0.00000,-0.00005,0.00000,0.00006,-0.00001,0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.20769361138871312

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.10083783e+07 2.32754355e+05 4.04243870e+04 2.96461279e+03
 1.65695806e+02 7.87090802e+00 2.82154850e-01 1.38331567e-32]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999729,-0.021377,-0.008887,-0.002444,-0.000595,-0.000138,-0.000030,-0.000005
1,0.020489,0.995952,-0.085317,-0.018939,-0.004548,-0.001050,-0.000228,-0.000040
2,0.010472,0.083741,0.994445,-0.061329,-0.013649,-0.003129,-0.000682,-0.000122
3,0.003416,0.023557,0.058446,0.995753,-0.065472,-0.014039,-0.003035,-0.000540
4,0.001034,0.007055,0.016662,0.063032,0.994371,-0.081504,-0.016507,-0.002920
5,0.000315,0.002149,0.005055,0.018380,0.077951,0.990114,-0.113477,-0.018803
6,0.000100,0.000687,0.001616,0.005857,0.024050,0.106678,0.977877,-0.178213
7,0.000037,0.000251,0.000591,0.002138,0.008760,0.037998,0.174919,0.983808


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00027118329625397397,
 0.004047871797165259,
 0.005555446221800819,
 0.004246866393708859,
 0.005628549043778519,
 0.009886399585076533,
 0.02212266081765024,
 0.016192180604453488]